In [1]:
import pandas as pd
import numpy as np

In [14]:
# read in the csv file
df = pd.read_csv("sample dataset/customer_details.csv", index_col='Customer ID')

df.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
Customer ID,,,,,,,,,,,,,,,,,
1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [ ]:
# take the user input:
def category_statistics(*args, df):
    with open("categorical_analysis.txt", "w") as file:
        for arg in args:
            if df[arg].dtype != 'object':
                raise ValueError("Please Enter a category feature / column")
            file.write(f"{arg} feature has the following\n")
            file.write(f"1- mode value: {df[arg].mode().values[0]}\n")
            file.write(f"2- the frequency of the mode value: {df.loc[df[arg] == df[arg].mode().values[0], arg].shape[0]}\n")
            file.write(f"3- count of non-null values: {df.loc[~df[arg].isna(), arg].shape[0]}\n")
            file.write(f"4- the number of unique values: {df[arg].nunique()}\n")
            if df[arg].nunique() <= 7:
                file.write(f"5- these unique values are : {[unique for unique in df[arg].unique().tolist()]}\n")
            file.write("--------------------------------------------\n")
category_statistics(*["Gender","Category","Location","Color"], df=df)

In [4]:
df.describe()

,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3900.000000
mean,44.068462,59.764359,3.749949,25.351538
std,15.207589,23.685392,0.716223,14.447125
min,18.000000,20.000000,2.500000,1.000000
25%,31.000000,39.000000,3.100000,13.000000
50%,44.000000,60.000000,3.700000,25.000000
75%,57.000000,81.000000,4.400000,38.000000
max,70.000000,100.000000,5.000000,50.000000


In [5]:
from scipy.stats import iqr
def count_outliers(col, df):
    q1 = np.percentile(df[col].dropna(), 25)
    q3 = np.percentile(df[col].dropna(), 75)
    iqr_value = iqr(df[col].dropna())
    lower_bound = q1 - (1.5 * iqr_value)
    upper_bound = q3 + (1.5 * iqr_value)

    low_outlier = df[df[col] < lower_bound][col].shape[0]
    high_outlier = df[df[col] > upper_bound][col].shape[0]

    total_outlier = low_outlier + high_outlier
    return (q1, q3, iqr_value, lower_bound, upper_bound, low_outlier, high_outlier, total_outlier)


In [ ]:
def numeric_statistics(*args, df):
    with open("numerical_analysis.txt", 'w') as file:
        for arg in args:
            if df[arg].dtype != 'int64':
                raise ValueError("Please Enter a numeric feature / column")
            file.write(f"{arg} feature has the following:\n")
            file.write(f"1- mean value: {round(df[arg].mean().tolist(),3)}\n")
            file.write(f"2- median value: {np.percentile(df[arg], 50)}\n")
            q1, q3, iqr_value, _, _, low_outlier, high_outlier, total_outlier = count_outliers(arg, df=df)
            file.write(f"3- first and the third quantiles: {[q1, q3]}\n")
            file.write(f"4- inter quantile range value: {iqr_value}\n")
            file.write(f"5- distributions:\n")
            if total_outlier in range(0, 15):
                file.write(f"total number of outliers: {total_outlier}\n")
            else:
                file.write(f"the total number of outliers: {total_outlier} with under the low bound: {low_outlier}, and above the upper bound: {high_outlier}\n")
            if df[arg].skew() < 0:
                file.write(f"this column is right skewed with value {df[arg].skew()}\n")
            elif df[arg].skew() > 0:
                file.write(f"this column is left skewed with value {df[arg].skew()}\n")
            else:
                file.write(f"this column is perfectly normal distributed\n")
            file.write("-" * 25)
numeric_statistics("Age", df=df)

In [7]:
df['Age'].dtype

dtype('int64')

In [12]:
print("Hello, this is a simple tool for analysis purposes.","please select the columns that presented in the datsets that you want to do analysis on.","check the new created files" ,sep='\n', end='.')
print()
print("make sure to copy the exact name with no mistake (case sensitive)")
print(df.columns.tolist())
print("press q to exit the loop")

selected_columns = {"numeric" : [], "category" : []}
patience = 0

while len(selected_columns) != len(df.columns.tolist()):
    usr_input = input("Please enter the feature name here:(or press q to exit)")
    if usr_input.lower() == 'q':
        break
    
    if usr_input not in df.columns.tolist():
        print("Please Enter a correct name of the features")
        patience += 1
        if patience == 3:
            print("ok you joking, get out of here")
            break

    else:
        if df[usr_input].dtype == "object":
            selected_columns['category'].append(usr_input)
        else:
            selected_columns['numeric'].append(usr_input)


category_statistics(*selected_columns['category'], df=df)
numeric_statistics(*selected_columns['numeric'], df=df)

Hello, this is a simple tool for analysis purposes.
please select the columns that presented in the datsets that you want to do analysis on.
check the new created files.
make sure to copy the exact name with no mistake (case sensitive)
['Age', 'Gender', 'Item Purchased', 'Category', 'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season', 'Review Rating', 'Subscription Status', 'Shipping Type', 'Discount Applied', 'Promo Code Used', 'Previous Purchases', 'Payment Method', 'Frequency of Purchases']
press q to exit the loop


In [13]:
print("thank you for using this simple tool!")

thank you for using this simple tool!
